In [ ]:
import numpy as np
import xarray as xr
from glob import glob
from sklearn.neighbors import BallTree
from skimage.morphology import skeletonize
import nc_time_axis
import cftime

In [ ]:
def prepro(ds):
    return ds.isel(y=slice(800, None))

In [ ]:
def extract_transect(lon_l, lat_l, dataset, n=None):
    """extract_transect
    Use a nearest neighbor lookup (BallTree) to find the indeces
    x, y in dataset that are closest to `lon_l`, `lat_l`.
    """
    coords = np.vstack([lat_l, lon_l]).T
    grid_coords = np.vstack([dataset.nav_lat.values.flat, dataset.nav_lon.values.flat]).T
    ball_tree = BallTree(np.deg2rad(grid_coords), metric='haversine')
    distances_radians, _ = ball_tree.query(np.deg2rad(coords), return_distance=True, breadth_first=True)
    index_y, index_x = np.unravel_index(_, dataset.nav_lat.shape)
    return index_y, index_x

In [ ]:
def extract_transects(transects_coords, ds_ref, ds_fut, var=None, outfile="output_{0}.nc"):
    """extract_trnasects
    Wrapper function for `extract_transect` that uses the x, y indices returned by `extract_transect` to
    get the data from the dataset, does some re-formatting and re-naming and writes the extracted
    transects to disk.
    """
    transects_data = []
    for counter, transect in enumerate(transects_coords):
        x = transect[0]
        y = transect[1]
        if len(x) != len(y):
            raise ValueError("Lenght of transect x and y should be identical") 
        index_y, index_x = extract_transect(x, y, ds_ref)
        ds_xi = xr.DataArray(index_x.ravel(), dims=["x_points"])
        ds_yi = xr.DataArray(index_y.ravel(), dims=["y_points"])
        ds_REF = ds_ref[var].assign_coords({"xt{0}".format(counter): ds_ref.x,
                                            "yt{0}".format(counter): ds_ref.y}).to_dataset().swap_dims({"x": "xt{0}".format(counter),
                                                                                                        "y": "yt{0}".format(counter)})
        transect_REF = ds_REF.isel({"xt{0}".format(counter): ds_xi,
                                    "yt{0}".format(counter): ds_yi}).rename({var: var+"_ref_t{0}".format(counter)})
        transect_REF = transect_REF.rename({"x_points": "x_points"+"_t{0}".format(counter),
                                            "y_points": "y_points"+"_t{0}".format(counter)})
        ds_FUT = ds_fut[var].assign_coords({"xt{0}".format(counter): ds_fut.x,
                                            "yt{0}".format(counter): ds_fut.y}).to_dataset().swap_dims({"x": "xt{0}".format(counter),
                                                                                                        "y": "yt{0}".format(counter)})
        transect_FUT = ds_FUT.isel({"xt{0}".format(counter): ds_xi,
                                    "yt{0}".format(counter): ds_yi}).rename({var: var+"_fut_t{0}".format(counter)})
        transect_FUT = transect_FUT.rename({"x_points": "x_points"+"_t{0}".format(counter),
                                            "y_points": "y_points"+"_t{0}".format(counter)})
        diag = xr.DataArray(np.arange(len(x)), dims="diag")
        diag_transect_REF = transect_REF.isel({"x_points"+"_t{0}".format(counter): diag, "y_points"+"_t{0}".format(counter): diag})
        diag_transect_FUT = transect_FUT.isel({"x_points"+"_t{0}".format(counter): diag, "y_points"+"_t{0}".format(counter): diag})
        transects_data.append([diag_transect_REF, diag_transect_FUT])
    transect_dataset = { "transect_{0}".format(ii) : xr.merge(transects_data[ii]) for ii in range(len(transects_data))}
    print("Storing data")
    [item.to_netcdf(outfile.format(key)) for key, item in transect_dataset.items()]
    return transect_dataset

Load grid and data files

In [ ]:
GRIDT_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-T_clim/*gridTclim.nc"
ICEMOD_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-icemod_clim/*icemodclim.nc"
GRIDT_data_filesFUT = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-T_clim/*gridTclim.nc"
ICEMOD_data_filesFUT = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-icemod_clim/*icemodclim.nc"

In [ ]:
grid_files = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_hgr.nc"
coords_file = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/coordinates_CREG12_lbclnk_noz_vh20160930.nc"
mask_file= "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mask.nc"

In [ ]:
grid = xr.open_mfdataset(grid_files, parallel=True, preprocess=prepro)
coords = xr.open_mfdataset(coords_file, parallel=True, preprocess=prepro)
mask = xr.open_mfdataset(mask_file, parallel=True, preprocess=prepro)

In [ ]:
GRIDT_REF = xr.open_mfdataset(GRIDT_data_filesREF, preprocess=prepro, parallel=True)
GRIDT_REF = GRIDT_REF.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
GRIDT_FUT = xr.open_mfdataset(GRIDT_data_filesFUT, preprocess=prepro, parallel=True)
GRIDT_FUT = GRIDT_FUT.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})

In [ ]:
ICEMOD_REF = xr.open_mfdataset(ICEMOD_data_filesREF, preprocess=prepro, parallel=True)
ICEMOD_REF = ICEMOD_REF.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
ICEMOD_FUT = xr.open_mfdataset(ICEMOD_data_filesFUT, preprocess=prepro, parallel=True)
ICEMOD_FUT = ICEMOD_FUT.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})

In [ ]:
mean_WpBp_REF = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/wp_bp_REF_clim.nc")
mean_WpBp_FUT = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/wp_bp_FUT_clim.nc")
mean_WpBp_REF = mean_WpBp_REF.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})
mean_WpBp_FUT = mean_WpBp_FUT.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})

In [ ]:
PVORT_REF = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-PVORT_clim/*PVORTclim.nc")
PVORT_FUT = xr.open_dataset("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-PVORT_clim/*PVORTclim.nc")
PVORT_REF = PVORT_REF.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})
PVORT_FUT = PVORT_FUT.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})

Define longitude (x) and latitude (y) of the transects to be extracted

In [ ]:
x_0 = np.linspace(-150,-150,400)
y_0 = np.linspace(70,90,400)

x_1 = np.linspace(0,0,300)
y_1 = np.linspace(90,75,300)

x_i = np.hstack((x_0,x_1))
y_i = np.hstack((y_0,y_1))

x_i2 = np.linspace(120,120,400)
y_i2 = np.linspace(90,73,400)

In [ ]:
transects = [[x_i,y_i],[x_i2,y_i2]]

Extract transects and save to disk with the functions defined above

In [ ]:
transects_dens = extract_transects(transects, GRIDT_REF, GRIDT_FUT, 'rhop_sig0', '/data0/project/drakkar/USERS/jrieck/transects/density_{0}.nc')
transects_ice = extract_transects(transects, ICEMOD_REF, ICEMOD_FUT, 'sithic', '/data0/project/drakkar/USERS/jrieck/transects/sithic_{0}.nc')
transects_WpBp = extract_transects(transects, mean_WpBp_REF, mean_WpBp_FUT, '__xarray_dataarray_variable__', '/data0/project/drakkar/USERS/jrieck/transects/Wp_Bp_{0}.nc')
transects_PV = extract_transects(transects, PVORT_REF, PVORT_FUT, 'vototvor', '/data0/project/drakkar/USERS/jrieck/transects/PVORT_{0}.nc')